In [1]:
!pip install skorch pytorch_lightning einops


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.datasets import MNIST


def create_dataloader(batch_size):
    data_train = torch.utils.data.DataLoader(
        MNIST(
            '~/mnist_data', train=True, download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])),
        batch_size=batch_size,
        shuffle=True
    )

    data_test = torch.utils.data.DataLoader(
        MNIST(
            '~/mnist_data', train=False, download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])),
        batch_size=batch_size,
        shuffle=True
    )

    return data_train, data_test


In [8]:
batch_size = 64

trainloader, testloader = create_dataloader(batch_size=batch_size)

In [9]:
from s4.skorch import S4Regression

model = S4Regression(
    max_epochs=5,
    optimizer=torch.optim.Adam,
    lr=0.001,
    batch_size=batch_size,
    criterion=nn.NLLLoss,
    d_input=1,
    d_output=783,
    d_model=512,
    n_layers=8,
    dropout=[0.2]*8,
    transposed=False,
    s4d=True,
)

In [10]:
model

<class 's4.skorch.S4Regression'>[uninitialized](
  module=SingleS4Regression(
    (encoder): Sequential(
      (0): Linear(in_features=1, out_features=512, bias=True)
      (1): GELU(approximate='none')
    )
    (decoder): Sequential(
      (0): Linear(in_features=512, out_features=783, bias=True)
      (1): Tanh()
    )
  ),
)

In [6]:
for idx, samples in enumerate(trainloader):
    print(f"{idx} ======")
    data, label = samples

    data = data.reshape(batch_size, -1)
    inputs = data[:, :-1].view(batch_size, -1, 1) #.cuda()
    targets = data[:, 1:].view(batch_size, -1).type(torch.LongTensor) #.cuda()

    # (batch_size, d_model, length)
    model.fit(inputs, targets)
    pred = model.predict(inputs)
    # loss = F.mse_loss(targets, pred)
    # print("loss ", loss.item())


0 ======
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.0003       -0.0008  8.5923
      2       -0.0008       -0.0018  7.0440
      3       -0.0018       -0.0029  6.7262
      4       -0.0029       -0.0039  6.7487
      5       -0.0040       -0.0050  6.7365
1 ======
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1       -0.0051       -0.0062  6.7107
      2       -0.0061       -0.0073  6.7212
      3       -0.0072       -0.0083  6.7184
      4       -0.0083       -0.0094  6.6795
      5       -0.0094       -0.0104  6.6822
2 ======
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1       -0.0105       -0.0115  6.7491
      2       -0.0115       -0.0126  6.7167
      3       -0.0126    